In [1]:
import os
import numpy as np 
import pandas as pd 

from glob import glob
from skimage.io import imread

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
stage1_test_path = '../input/stage1_test'
stage1_test_old_path = '../input/stage1_test_old'

In [3]:
stage1_test_images = glob(stage1_test_path + '/*/images/*.png')
stage1_test_old_images = glob(stage1_test_old_path + '/*/images/*.png')
set(stage1_test_images) == set(stage1_test_old_images)

False

In [4]:
stage1_test_images = [image.split('test')[-1] for image in stage1_test_images]
stage1_test_old_images = [image.split('test_old')[-1] for image in stage1_test_old_images]
set(stage1_test_images) == set(stage1_test_old_images)

True

In [5]:
stage1_solution_df = pd.read_csv('../input/stage1_solution.csv')
stage1_solution_df.head(2)

,ImageId,EncodedPixels,Height,Width,Usage
0,0114f484a16c152baa2d82fdd43740880a762c93f436c8...,31632 6 31886 10 32139 14 32394 15 32649 17 32...,256,256,Public
1,0114f484a16c152baa2d82fdd43740880a762c93f436c8...,50280 7 50534 10 50788 13 51043 14 51298 16 51...,256,256,Public


In [6]:
from skimage.morphology import label # label regions
def rle_encoding(x):
    '''
    x: numpy array of shape (height, width), 1 - mask, 0 - background
    Returns run length as list
    '''
    dots = np.where(x.T.flatten()==1)[0] # .T sets Fortran order down-then-right
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b+1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

In [7]:
test_image_ids = np.unique(stage1_solution_df["ImageId"])

In [8]:
for image_id in test_image_ids:
    df = stage1_solution_df[stage1_solution_df['ImageId'] == image_id]
    df['EncodedPixels'] = df['EncodedPixels'].map(lambda ep: [int(x) for x in ep.split(' ')])

    mask_paths = glob(stage1_test_path + '/' + image_id + '/masks/*.png')
    rles = []
    for path in mask_paths:
        mask = imread(path)
        mask = (mask > 0).astype(np.int8)
        rle = rle_encoding(mask)
        rles.append(rle)
    assert(sorted(rles) == sorted(list(df['EncodedPixels'])))
    

/home/jun/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
